In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import torch
from RAVEN.pe.appr_taylor_utils import *

# Test for RoundingNoGrad

In [23]:
input = torch.tensor([0.51, 0.49, -0.51, -0.49])
print("ceil :", RoundingNoGrad.apply(input, "ceil"))
print("floor:", RoundingNoGrad.apply(input, "floor"))
print("round:", RoundingNoGrad.apply(input, "round"))

ceil : tensor([1., 1., -0., -0.])
floor: tensor([ 0.,  0., -1., -1.])
round: tensor([ 1.,  0., -1., -0.])


# Test for Trunc

In [27]:
input = torch.tensor([0.751, 0.749, -0.751, -0.749, 0.759, 0.741, -0.759, -0.741])
print(Trunc(input, intwidth=1, fracwidth=2, rounding="ceil"))
print(Trunc(input, intwidth=1, fracwidth=2, rounding="floor"))
print(Trunc(input, intwidth=1, fracwidth=2, rounding="round"))

tensor([ 1.0000,  0.7500, -0.7500, -0.5000,  1.0000,  0.7500, -0.7500, -0.5000])
tensor([ 0.7500,  0.5000, -1.0000, -0.7500,  0.7500,  0.5000, -1.0000, -0.7500])
tensor([ 0.7500,  0.7500, -0.7500, -0.7500,  0.7500,  0.7500, -0.7500, -0.7500])


# Test for Appr_Taylor

In [43]:
a = torch.arange(0, 1, 0.001)
a_int = a.floor()

precise = torch.exp(a)

point = torch.tensor(0.)
scale = torch.exp(a_int + point)
const = torch.tensor(1.0)
input = a - point

coeff = [1/1, 1/2, 1/4, 1/8, 1/16, 1/32]
power = [1  ,   2,   3,   4,    5,    5]
sign  = [1  ,   1,   1,  -1,    1,    1]

approximate = Appr_Taylor(scale, 
                          const, 
                          input, 
                          coeff, 
                          power, 
                          sign, 
                          fxp=True, 
                          intwidth=7, 
                          fracwidth=8, 
                          rounding="round", 
                          keepwidth=True)

error = (approximate - precise) / precise
print("min error rate:", error.min())
print("max error rate:", error.max())
print("avg error rate:", error.mean())
print("rms error rate:", error.mul(error).mean().sqrt())

min error rate: tensor(-0.0043)
max error rate: tensor(0.0060)
avg error rate: tensor(0.0007)
rms error rate: tensor(0.0018)
